In [1]:
# Import all the dependencies

import nvidia
import os
import time
import torch
import transformers

from datasets import load_dataset
from random import randint
from vllm import LLM, SamplingParams

In [2]:
cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir

In [3]:
# Load dataset from the hub
test_dataset = load_dataset("samsum", split="test")

In [4]:
# Load the model to vLLM.
llm = LLM(model="/mnt/falcon_7b_model_adapter",trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type RefinedWebModel to instantiate a model of type falcon. This is not supported for all configurations of models and can yield errors.


INFO 08-18 20:32:26 llm_engine.py:70] Initializing an LLM engine with config: model='/mnt/falcon_7b_model_adapter', tokenizer='/mnt/falcon_7b_model_adapter', tokenizer_mode=auto, trust_remote_code=True, dtype=torch.float16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, seed=0)
INFO 08-18 20:32:45 llm_engine.py:196] # GPU blocks: 43795, # CPU blocks: 32768


In [5]:
# select a random test sample
# sample = test_dataset[randint(0, len(test_dataset))]

# Change the index to select a different sample
sample = test_dataset[5]

# format sample
prompt_template = f"Summarize the chat dialogue:\n{{dialogue}}\n---\nSummary:\n"

test_sample = prompt_template.format(dialogue=sample["dialogue"])

print(test_sample)

Summarize the chat dialogue:
Benjamin: Hey guys, what are we doing with the keys today?
Hilary: I've got them. Whoever wants them can meet me at lunchtime or after
Elliot: I'm ok. We're meeting for the drinks in the evening anyway and I guess we'll be going back to the apartment together?
Hilary: Yeah, I guess so
Daniel: I'm with Hilary atm and won't let go of her for the rest of the day, so any option you guys choose is good for me
Benjamin: Hmm I might actually pass by at lunchtime, take the keys and go take a nap. I'm sooo tired after yesterday
Hilary: Sounds good. We'll be having lunch with some French people (the ones who work on the history of food in colonial Mexico - I already see you yawning your head off)
Benjamin: YAAAAWN 🙊 Where and where are you meeting?
Hilary: So I'm meeting them at the entrance to the conference hall at 2 pm and then we'll head to this place called La Cantina. Italian cuisine, which is quite funny, but that's what they've chosen
Benjamin: Interesting 😱 

In [6]:
max_tokens = 50

sampling_params = SamplingParams(
        temperature=1.0,
        top_p=1,
        max_tokens=max_tokens,
    )
start_time = time.perf_counter()
outputs = llm.generate(test_sample, sampling_params)
end_time = time.perf_counter()
generated_text = outputs[0].outputs[0].text
print(f"{generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]

'Hilary is going to lunch. Benjamin might pass by after lunchtime to take the keys from Hilary. Hilary is meeting French people to talk about colonial food and she is going to La Cantina for Italian cuisine afterwards. Elliot will meet their French guests'


In [7]:
print(f'\n vLLM took {round(end_time - start_time, 3)} sec and generated {round(max_tokens / (end_time - start_time),3)}  tokens/sec')


 vLLM took 1.781 sec and generated 28.076  tokens/sec


In [ ]:
%%timeit -n 1 -r 30
outputs = llm.generate(test_sample, sampling_params)
generated_text = outputs[0].outputs[0].text